In [4]:
import os
import sys

# If the following line throws an error, do `pip install -e ./` from the base repo dir
import utils

# Change to match data filepath on local
base_fp = r'D:\Documents\MIDS\W281\2023-mids-w81-final-project-dataset\256x256'

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from typing import Iterable, Tuple, List
from pathlib import Path


In [ ]:
def build_model(optimizer: str,
        conv_layers: List[int],
        hidden_layers: List[int], 
        activation: str = 'relu',
        dropout_rate: float = 0.0,
        regularize: bool = False,
        normalize: bool = False) -> keras.Sequential:
    ''' Build CNN model with given parameters'''
    
    tf.keras.backend.clear_session()
    model = keras.Sequential()

    regularizer = tf.keras.regularizers.l2(l=0.01) if regularize else None

    for i, layer in enumerate(conv_layers):
        # Convolutional layer and maxpool layer
        if i == 0:
            model.add(keras.layers.Conv2D(layer,(3,3),activation=activation,input_shape=IMAGE_SHAPE, kernel_regularizer=regularizer))
        else:
            model.add(keras.layers.Conv2D(layer,(3,3),activation=activation, kernel_regularizer=regularizer))
        model.add(keras.layers.MaxPool2D(2,2))
        if dropout_rate > 0:
            model.add(keras.layers.Dropout(dropout_rate))

    # This layer flattens the resulting image array to 1D array
    model.add(keras.layers.Flatten())

    for i, hidden in enumerate(hidden_layers):
        model.add(keras.layers.Dense(hidden,activation=activation))
        if dropout_rate > 0:
            model.add(keras.layers.Dropout(dropout_rate))
        if i != 0 and i != len(hidden_layers) - 1 and normalize:
            model.add(keras.layers.BatchNormalization())


    # Output layer with single neuron between 0 and 1
    model.add(keras.layers.Dense(1,activation='sigmoid'))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model
